In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
from sklearn.compose import ColumnTransformer

In [4]:
# Loading All the models
model = load_model('pickle_files/NN_model.h5')

# Loading Scaler and Encoder models
with open('pickle_files/labelencoder.pkl', 'rb') as file:
    le_gender = pickle.load(file)

with open('pickle_files/ohe.pkl', 'rb') as file:
    ohe_geo = pickle.load(file)

with open('pickle_files/scaling.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [74]:
input_querie = {
    'CreditScore' : 600, 
    'Geography' : 'France', 
    'Gender' : 'Male', 
    'Age' : 40, 
    'Tenure' : 2, 
    'Balance' : 70000,
    'NumOfProducts' : 3, 
    'HasCrCard' : 1, 
    'IsActiveMember' : 0, 
    'EstimatedSalary' : 60000,
}

In [90]:
input_querie.keys()

dict_keys(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])

In [21]:
# np.array(list(input_querie.values())).reshape(1,-1)

array([['600', 'France', 'Male', '40', '2', '70000', '3', '1', '0',
        '60000']], dtype='<U11')

In [24]:
# input_df = pd.DataFrame(np.array(list(input_querie.values())).reshape(1,-1), columns=input_querie.keys())

In [76]:
input_df = pd.DataFrame(input_querie, index=[0])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,2,70000,3,1,0,60000


In [77]:
input_df['Gender'] = le_gender.transform(input_df['Gender'])

In [78]:
ohe_col = ohe_geo.transform(input_df[['Geography']]).toarray()
ohe_df = pd.DataFrame(ohe_col, columns=ohe_geo.get_feature_names_out())

In [79]:
input_df = pd.concat([input_df.drop('Geography', axis=1), ohe_df], axis=1)

In [80]:
input_df.reset_index(drop=True)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,2,70000,3,1,0,60000,1.0,0.0,0.0


In [81]:
scaled_input_df = scaler.transform(input_df)

In [86]:
scaled_input_df

array([[-0.53598516,  0.91324755,  0.10479359, -1.04241787, -0.09770129,
         2.53355998,  0.64920267, -1.02583358, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [82]:
scaled_input_df.shape

(1, 12)

In [83]:
y_pred = model.predict(scaled_input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [87]:
y_pred

array([[0.96407986]], dtype=float32)

In [88]:
def class_returner(value):
    if value[0][0] > 0.5:
        print("The person will likely to Churn")
    
    else:
        print('The person will not likely to Churn')

In [89]:
class_returner(y_pred)

The person will likely to Churn


In [92]:
ohe_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [93]:
le_gender.classes_

array(['Female', 'Male'], dtype=object)